<div align="center"> 
    <hr>
  <font size="6"><b>Wallmart - Amazon</b> </font>
   <hr>
</div>

In [1]:
!python --version

Python 3.9.1


In [1]:
import os
import sys
import pandas as pd
import networkx
from networkx import draw, Graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from pyjedai.utils import (
    text_cleaning_method,
    print_clusters,
    print_blocks,
    print_candidate_pairs
)
from pyjedai.evaluation import Evaluation, write
from pyjedai.datamodel import Data
# d1 = pd.read_csv("./data/test/ccer/abt_100.csv", sep='|', engine='python', na_filter=False).astype(str)
# d2 = pd.read_csv("./data/test/ccer/buy_100.csv", sep='|', engine='python', na_filter=False).astype(str)
# gt = pd.read_csv("./data/test/ccer/gt_100.csv", sep='|', engine='python')

d1 = pd.read_csv("./data/ccer/D8/amazon.csv", sep='|', engine='python', na_filter=False).astype(str)
d2 = pd.read_csv("./data/ccer/D8/walmart.csv", sep='|', engine='python', na_filter=False).astype(str)
gt = pd.read_csv("./data/ccer/D8/gt.csv", sep='|', engine='python')


data = Data(dataset_1=d1,
            # attributes_1=['id','name','description'],
            id_column_name_1='id',
            dataset_2=d2,
            # attributes_2=['id','name','description'],
            id_column_name_2='id',
            ground_truth=gt)

# Block Building

## FAISS

faiss.IndexIVFFlat is an implementation of an inverted file index with coarse quantization. This index is used to efficiently search for nearest neighbors of a query vector in a large dataset of vectors. Here's a brief explanation of the parameters used in this index:

- `d`: The dimension of the input vectors. Each input vector should have d elements.
- `nlist`: The number of cells (or clusters) to divide the dataset into. This determines the number of inverted lists in the index, and hence the maximum number of vectors that can be stored in the index. Larger values of nlist will generally result in more accurate search results, but may also increase the memory requirements of the index and the time required to perform searches.
- `quantizer`: An instance of a quantizer object that is used to compress the vectors before they are added to the index. The quantizer object should implement a train method that learns the quantization function from a set of input vectors, and a compress method that compresses a vector into an integer code. FAISS provides several built-in quantizer objects, such as IndexFlatL2 and IndexScalarQuantizer.
- `metric`: The distance metric used to measure the similarity between vectors. The metric should be one of the strings defined in the faiss.METRIC_TYPES list, such as "l2" for Euclidean distance or "ip" for inner product.
- `nprobe`: The number of cells (or clusters) to search during a query. This parameter determines the tradeoff between accuracy and speed during searches. Larger values of nprobe will generally result in more accurate search results, but may also increase the time required to perform searches.



In [2]:
from pyjedai.vector_based_blocking import EmbeddingsNNBlockBuilding

In [3]:
emb = EmbeddingsNNBlockBuilding(vectorizer='st5',
                                similarity_search='faiss')
blocks = emb.build_blocks(data, top_k=10, num_of_clusters=10)

Embeddings-NN Block Building [st5, faiss]:   0%|          | 0/24628 [00:00<?, ?it/s]

Device selected:  cpu
True
22074


In [4]:
emb.evaluate(blocks, with_classification_report=True, with_stats=True)

KeyError: '10706'

# Entity Matching

In [ ]:
from pyjedai.matching import EntityMatching

In [ ]:
EM = EntityMatching(metric='cosine_vector_similarity',
                    similarity_threshold=0)

pairs_graph = EM.predict(blocks,
                         data,
                         tqdm_disable=True,
                         vectors_d1=emb.vectors_1,
                         vectors_d2=emb.vectors_2)

In [ ]:
draw(pairs_graph)

In [ ]:
EM.evaluate(pairs_graph)

In [ ]:
EM.plot_distribution_of_scores()

# Entity Clustering

In [ ]:
from pyjedai.clustering import ConnectedComponentsClustering, UniqueMappingClustering

In [ ]:
ccc = ConnectedComponentsClustering()
clusters = ccc.process(pairs_graph, data)

In [ ]:
ccc.report()

In [ ]:
ccc.evaluate(clusters, with_classification_report=True)

In [ ]:
from pyjedai.clustering import ConnectedComponentsClustering, UniqueMappingClustering
umc = UniqueMappingClustering()
clusters = umc.process(pairs_graph, data)

In [ ]:
umc.evaluate(clusters, with_classification_report=True)